<a href="https://colab.research.google.com/github/Hidestament/AtCoder/blob/main/ABC/ABC202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ABC202
https://atcoder.jp/contests/abc202

## A - Three Dice

In [ ]:
a,b,c = map(int, input().split())
print(21 - a - b - c)

1 4 3
13


## B - 180°


In [ ]:
S = str(input())
ans = ""
for i in S:
  if i == "6":
    ans = "9" + ans
  elif i == "9":
    ans = "6" + ans
  else:
    ans = i + ans
print(ans)

0601889
6881090


## C - Made Up

In [ ]:
from collections import Counter
N = int(input())
A = list(map(int, input().split()))
B = list(map(int, input().split()))
C = list(map(int, input().split()))

3
1 2 2
3 1 2
2 3 2


In [ ]:
B_C = [0 for j in range(len(C))]
for j in range(len(C)):
  B_C[j] = B[C[j] - 1]
B_C_dict = Counter(B_C)

In [ ]:
ans = 0
for a in A:
  ans += B_C_dict[a]
print(ans)

## D - aab aba baa

#### 解法1 : ダメ解法
愚直にやると全く終わらない

In [ ]:
A, B, K = map(int, input().split())

30 30 118264581564861424


In [ ]:
temp = 0
for now in range(0, 2**(A+B)):
  now = str(bin(now))[2:].zfill(A+B)
  if now.count("0") == A and now.count("1") == B:
    temp += 1
  if temp == K:
    print(now.replace("0", "a").replace("1", "b"))

#### 解法2 : 先頭から a or b を決めていく
辞書順の並び方を求めるときは, 先頭から決めていくのが定石.

辞書順では, a -> b の順で進んでいく. ここで, 先頭がaである場合の並べ方の総数をnとすると, $n < k $ であるなら, 先頭がaでは $k$番目の順番を表現することができない.

よって, $n < k $なら先頭の文字は b となる.

逆に$n \geq k$ なら$k$番目の順番を先頭が$a$で表現することができる.

よって, この場合先頭の文字はaとなる.

- 並べ方の総数をどう計算するか

今, 使用していないaとbの個数が, それぞれA, Bであるとする.

このとき, 先頭をaと固定して, 残りの全てを並べるので, 
$$
_{A-1+B} C _{B} * _{A-1} C _{A-1} = _{A-1+B} C _{B}
$$
となる. これは, A+B-1の場所から, bの場所をB個選ぶ組合せである.

以下ではmath.factorial() を使ったが, パスカルの3角形を使って事前に計算すると良いらしい.

In [ ]:
from math import factorial
A, B, K = map(int, input().split())
ans = [0 for _ in range(A+B)]
pattern = 0
for i in range(A+B):
  res_a = A - ans.count("a") - 1
  res_b = B - ans.count("b")

  if res_a < 0:
    ans[i] = "b"
  elif res_b < 0:
    ans[i] = "a"
  else:
    temp = factorial(res_a + res_b) // factorial(res_a) // factorial(res_b)
  
    if pattern + temp < K:
      ans[i] = "b"
      pattern += temp
    else:
      ans[i] = "a"
print("".join(ans))

#### 解法2' : パスカルの3角形
上のcombinationsのところをパスカルの３角形でやってみる

In [44]:
A, B, K = map(int, input().split())

2 2 4


In [45]:
combinations = [[0 for _ in range(A+B+1)] for __ in range(A+B+1)]
combinations[0][0] = 1
for i in range(A+B):
  for j in range(i+1):
    combinations[i+1][j] += combinations[i][j]
    combinations[i+1][j+1] +=  combinations[i][j]

In [46]:
ans = [0 for _ in range(A+B)]
for i in range(A+B):
  res_a = A - ans.count("a") - 1
  res_b = B - ans.count("b") - 1

  if res_a < 0:
    ans[i] = "b"
  elif res_b <= 0:
    ans[i] = "a"
  else:
    temp = combinations[res_a + res_b][res_a]
    if temp < K:
      ans[i] = "b"
      K -= temp
    else:
      ans[i] = "a"

In [49]:
print("".join(ans))

baab
